# Protein ID Mapping to HGNC Symbols

This notebook demonstrates the process of mapping protein IDs to HGNC (HUGO Gene Nomenclature Committee) symbols. The workflow includes loading protein (or phosphosite) data, extracting protein IDs, mapping these IDs using different databases, and updating our dataset with the new identifiers.

## Workflow Steps:
1. Setup and import necessary libraries and modules.
2. Load the dataset containing protein IDs.
3. Extract protein IDs from the dataset.
4. Map protein IDs to HGNC symbols using UniProt and HGNC databases.
5. Update the dataset with mapped HGNC symbols.
6. Analyze and summarize the results.

Let's begin by setting up our environment.

In [1]:
# Import necessary libraries
import pandas as pd
import os
import logging

# Import functions from the id_mapping module
import src.id_mapping as idm

# Configure logging
logging.basicConfig(level=logging.INFO)

# Set working directory
os.chdir('../')

## Loading Protein Data

We start by loading our dataset containing Area Under the Peak (AUP) data from LC-MS/MS phosphoproteomics experiments. Rows represent phosphosites (or features), and columns represent samples. Understanding the structure of our data is crucial for the subsequent mapping process.

In [2]:
# Load the dataset
aup = pd.read_csv('resources/raw_data/phosphodata_aup.tsv', sep='\t', index_col=0)

# Display the first few rows of the dataset
aup.head()

,b1790p079_PP1_MCF7_AZD1480_r1,b1790p079_PP1_MCF7_AZD1480_r2,b1790p079_PP1_MCF7_AZD5363_r1,b1790p079_PP1_MCF7_AZD5363_r2,b1790p079_PP1_MCF7_BX_r1,b1790p079_PP1_MCF7_BX_r2,b1790p079_PP1_MCF7_CHIR_r1,b1790p079_PP1_MCF7_CHIR_r2,b1790p079_PP1_MCF7_DMSO_r1,b1790p079_PP1_MCF7_DMSO_r2,...,b1790p077_PP2_MCF7_KU_r1,b1790p077_PP2_MCF7_KU_r2,b1790p077_PP2_MCF7_PD_r1,b1790p077_PP2_MCF7_PD_r2,b1790p077_PP2_MCF7_PF_r1,b1790p077_PP2_MCF7_PF_r2,b1790p077_PP2_MCF7_TORIN_r1,b1790p077_PP2_MCF7_TORIN_r2,b1790p077_PP2_MCF7_TRAMETINIB_r1,b1790p077_PP2_MCF7_TRAMETINIB_r2
Substrate,,,,,,,,,,,,,,,,,,,,,
1A24_HUMAN(S356),2.692968e+08,3.502752e+08,2.339065e+08,1.473238e+08,2.292088e+08,1.992612e+08,2.226755e+08,2.031111e+08,1.724312e+08,1.952799e+08,...,1.942673e+08,2.446224e+08,1.228907e+08,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,4.607186e+08
1A24_HUMAN(S359),2.692968e+08,3.502752e+08,2.339065e+08,1.473238e+08,2.292088e+08,1.992612e+08,2.226755e+08,2.031111e+08,1.724312e+08,1.952799e+08,...,1.942673e+08,2.446224e+08,1.228907e+08,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,4.607186e+08
1B39_HUMAN(M1),7.695947e+08,5.231202e+08,1.278819e+09,7.016728e+08,6.317868e+07,3.288562e+08,7.672546e+08,8.864559e+08,3.021734e+09,1.579953e+09,...,0.000000e+00,0.000000e+00,5.279555e+08,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,1.658262e+08
1B39_HUMAN(M4),7.695947e+08,5.231202e+08,1.278819e+09,7.016728e+08,6.317868e+07,3.288562e+08,7.672546e+08,8.864559e+08,3.021734e+09,1.579953e+09,...,0.000000e+00,0.000000e+00,5.279555e+08,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,1.658262e+08
AAAS(S495),2.664712e+09,3.635279e+09,3.285959e+09,2.743536e+09,3.822861e+09,4.393195e+09,3.287022e+09,3.540691e+09,2.247902e+09,6.012226e+09,...,8.963979e+09,8.351130e+09,9.330759e+09,7.767012e+09,7.900387e+09,7.211204e+09,8.041161e+09,8.921806e+09,8.499730e+09,7.352127e+09


## Extracting Protein IDs

Because the identifiers in our phosphoproteomics dataset are phosphosite IDs, we need to extract unique protein IDs from our dataset (indices) before mapping. These IDs will be used to fetch corresponding HGNC symbols. This is handled by the respective mapping functions (get_uniprot_id_map, get_hgnc_id_map) internally, but here is an example:

In [3]:
# Extract unique protein IDs
extracted_ids = idm.extract_protein_ids(aup.index)

# Display extracted IDs
print('Phosphosite IDs: ', aup.index.to_list()[:5])
print('Protein IDs: ', extracted_ids[:5])

Phosphosite IDs:  ['1A24_HUMAN(S356)', '1A24_HUMAN(S359)', '1B39_HUMAN(M1)', '1B39_HUMAN(M4)', 'AAAS(S495)']
Protein IDs:  ['1A24_HUMAN', '1B39_HUMAN', 'AAAS', 'AAGAB', 'AAK1']


## 1. Mapping Protein IDs to HGNC Symbols Using UniProt (UniProtMapper)

We utilize the UniProt database to map our protein IDs to HGNC symbols. This mapping is crucial for standardizing our dataset. We use the UniProtMapper, a Python wrapper for UniProt's Retrieve/ID Mapping RESTful API. 

In [4]:
# Map protein IDs using UniProt
uniprot_map = idm.get_uniprot_id_map(
    aup.index.to_list(), 
    from_id="UniProtKB_AC-ID", 
    to_id="Gene_Name", 
    tool='uniprot_protmapper', 
    phosphosites=True, 
    return_df=True)

# Show a snippet of the mapping results
aup.head()

Setting fields to `None` to retrieve all available fields...


Fetched: 3 / 500
Fetched: 2 / 500
Fetched: 6 / 500
Fetched: 4 / 500
Fetched: 0 / 500
Fetched: 3 / 500
Fetched: 3 / 500
Fetched: 2 / 500


INFO:root:
Mapped 30 out of 4593 proteins from UniProtKB_AC-IDs to Gene_Names.


Fetched: 5 / 500
Fetched: 2 / 93


,b1790p079_PP1_MCF7_AZD1480_r1,b1790p079_PP1_MCF7_AZD1480_r2,b1790p079_PP1_MCF7_AZD5363_r1,b1790p079_PP1_MCF7_AZD5363_r2,b1790p079_PP1_MCF7_BX_r1,b1790p079_PP1_MCF7_BX_r2,b1790p079_PP1_MCF7_CHIR_r1,b1790p079_PP1_MCF7_CHIR_r2,b1790p079_PP1_MCF7_DMSO_r1,b1790p079_PP1_MCF7_DMSO_r2,...,b1790p077_PP2_MCF7_KU_r1,b1790p077_PP2_MCF7_KU_r2,b1790p077_PP2_MCF7_PD_r1,b1790p077_PP2_MCF7_PD_r2,b1790p077_PP2_MCF7_PF_r1,b1790p077_PP2_MCF7_PF_r2,b1790p077_PP2_MCF7_TORIN_r1,b1790p077_PP2_MCF7_TORIN_r2,b1790p077_PP2_MCF7_TRAMETINIB_r1,b1790p077_PP2_MCF7_TRAMETINIB_r2
Substrate,,,,,,,,,,,,,,,,,,,,,
1A24_HUMAN(S356),2.692968e+08,3.502752e+08,2.339065e+08,1.473238e+08,2.292088e+08,1.992612e+08,2.226755e+08,2.031111e+08,1.724312e+08,1.952799e+08,...,1.942673e+08,2.446224e+08,1.228907e+08,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,4.607186e+08
1A24_HUMAN(S359),2.692968e+08,3.502752e+08,2.339065e+08,1.473238e+08,2.292088e+08,1.992612e+08,2.226755e+08,2.031111e+08,1.724312e+08,1.952799e+08,...,1.942673e+08,2.446224e+08,1.228907e+08,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,4.607186e+08
1B39_HUMAN(M1),7.695947e+08,5.231202e+08,1.278819e+09,7.016728e+08,6.317868e+07,3.288562e+08,7.672546e+08,8.864559e+08,3.021734e+09,1.579953e+09,...,0.000000e+00,0.000000e+00,5.279555e+08,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,1.658262e+08
1B39_HUMAN(M4),7.695947e+08,5.231202e+08,1.278819e+09,7.016728e+08,6.317868e+07,3.288562e+08,7.672546e+08,8.864559e+08,3.021734e+09,1.579953e+09,...,0.000000e+00,0.000000e+00,5.279555e+08,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,1.658262e+08
AAAS(S495),2.664712e+09,3.635279e+09,3.285959e+09,2.743536e+09,3.822861e+09,4.393195e+09,3.287022e+09,3.540691e+09,2.247902e+09,6.012226e+09,...,8.963979e+09,8.351130e+09,9.330759e+09,7.767012e+09,7.900387e+09,7.211204e+09,8.041161e+09,8.921806e+09,8.499730e+09,7.352127e+09


## Updating the DataFrame with Mapped IDs

After mapping our protein IDs to HGNC symbols, we update our original dataset with these new identifiers.

In [5]:
# Update the DataFrame with HGNC symbols
aup_m1 = idm.index_ids_to_hgnc_symbols(aup, database='uniprot_protmapper', phosphosites=True)

# Display the updated DataFrame
aup_m1.head()

Setting fields to `None` to retrieve all available fields...


Fetched: 3 / 500
Fetched: 2 / 500
Fetched: 6 / 500
Fetched: 4 / 500
Fetched: 0 / 500
Fetched: 3 / 500
Fetched: 3 / 500
Fetched: 2 / 500


INFO:root:
Mapped 30 out of 4593 proteins from UniProtKB_AC-IDs to Gene_Names.
INFO:root:Transformed 36 out of 14448 phosphosites.


Fetched: 5 / 500
Fetched: 2 / 93


,b1790p079_PP1_MCF7_AZD1480_r1,b1790p079_PP1_MCF7_AZD1480_r2,b1790p079_PP1_MCF7_AZD5363_r1,b1790p079_PP1_MCF7_AZD5363_r2,b1790p079_PP1_MCF7_BX_r1,b1790p079_PP1_MCF7_BX_r2,b1790p079_PP1_MCF7_CHIR_r1,b1790p079_PP1_MCF7_CHIR_r2,b1790p079_PP1_MCF7_DMSO_r1,b1790p079_PP1_MCF7_DMSO_r2,...,b1790p077_PP2_MCF7_KU_r1,b1790p077_PP2_MCF7_KU_r2,b1790p077_PP2_MCF7_PD_r1,b1790p077_PP2_MCF7_PD_r2,b1790p077_PP2_MCF7_PF_r1,b1790p077_PP2_MCF7_PF_r2,b1790p077_PP2_MCF7_TORIN_r1,b1790p077_PP2_MCF7_TORIN_r2,b1790p077_PP2_MCF7_TRAMETINIB_r1,b1790p077_PP2_MCF7_TRAMETINIB_r2
1A24_HUMAN(S356),2.692968e+08,3.502752e+08,2.339065e+08,1.473238e+08,2.292088e+08,1.992612e+08,2.226755e+08,2.031111e+08,1.724312e+08,1.952799e+08,...,1.942673e+08,2.446224e+08,1.228907e+08,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,4.607186e+08
1A24_HUMAN(S359),2.692968e+08,3.502752e+08,2.339065e+08,1.473238e+08,2.292088e+08,1.992612e+08,2.226755e+08,2.031111e+08,1.724312e+08,1.952799e+08,...,1.942673e+08,2.446224e+08,1.228907e+08,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,4.607186e+08
1B39_HUMAN(M1),7.695947e+08,5.231202e+08,1.278819e+09,7.016728e+08,6.317868e+07,3.288562e+08,7.672546e+08,8.864559e+08,3.021734e+09,1.579953e+09,...,0.000000e+00,0.000000e+00,5.279555e+08,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,1.658262e+08
1B39_HUMAN(M4),7.695947e+08,5.231202e+08,1.278819e+09,7.016728e+08,6.317868e+07,3.288562e+08,7.672546e+08,8.864559e+08,3.021734e+09,1.579953e+09,...,0.000000e+00,0.000000e+00,5.279555e+08,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,1.658262e+08
AAAS(S495),2.664712e+09,3.635279e+09,3.285959e+09,2.743536e+09,3.822861e+09,4.393195e+09,3.287022e+09,3.540691e+09,2.247902e+09,6.012226e+09,...,8.963979e+09,8.351130e+09,9.330759e+09,7.767012e+09,7.900387e+09,7.211204e+09,8.041161e+09,8.921806e+09,8.499730e+09,7.352127e+09


## 2. Mapping Protein IDs to HGNC Symbols Using UniProt (UniProt Search)

Despite the previous ID mapping step, some UniProt entries remain unmapped, likely because they are historical entry names, which makes them inaccessible via the standard UniProt ID Mapping function. However, a conventional UniProt search includes entry history. By identifying all IDs still marked with the UniProt entry tag (_HUMAN), we can employ a UniProt search to locate these entries. We then extract the gene name associated with the first search result.

In [6]:
# Identify UniProt IDs that have not been mapped and contain the "_HUMAN" tag.
filtered_ids = [id_ for id_ in aup_m1.index.to_list() if '_HUMAN' in id_]
print(f"Sample of filtered IDs for further mapping: {filtered_ids[:5]}")

# Use the UniProt search to map these filtered IDs to gene names.
uniprot_map = idm.get_uniprot_id_map(
    filtered_ids, 
    from_id="UniProtKB_AC-ID", 
    to_id="Gene_Name", 
    tool='uniprot_search', 
    phosphosites=True, 
    return_df=True
)
# Display the first few mappings to verify the process
uniprot_map.head()

Sample of filtered IDs for further mapping: ['1A24_HUMAN(S356)', '1A24_HUMAN(S359)', '1B39_HUMAN(M1)', '1B39_HUMAN(M4)', 'AIM1_HUMAN(S103)']


ERROR:root:
FA21D_HUMAN doesn't exist in UniProtKB database.


INFO:root:
Mapped 105 out of 114 proteins from UniProtKB_AC-IDs to Gene_Names.


,From,To
1,1B39_HUMAN,SYNDIG1
2,AIM1_HUMAN,CRYBG1
3,ASUN_HUMAN,INTS13
4,BICR1_HUMAN,BICDL1
5,CA106_HUMAN,INAVA


In [7]:
# Preparing to update our DataFrame with the new mappings.
# Splitting the original DataFrame into two parts:
# 1. Entries corresponding to the filtered IDs for updating.
# 2. The rest of the entries that don't need updating.
aup_m2a = aup_m1.loc[filtered_ids]
aup_m2b = aup_m1.drop(filtered_ids)

# Update the entries with the newly mapped HGNC symbols using UniProt search results.
aup_m2a = idm.index_ids_to_hgnc_symbols(aup_m2a, database='uniprot_search', phosphosites=True)

# Reassemble the DataFrame by concatenating the updated part with the unchanged part.
aup_m2 = pd.concat([aup_m2a, aup_m2b])

# Display the updated DataFrame to confirm the successful mapping and integration.
aup_m2.head()

ERROR:root:
FA21D_HUMAN doesn't exist in UniProtKB database.


INFO:root:
Mapped 105 out of 114 proteins from UniProtKB_AC-IDs to Gene_Names.
INFO:root:Transformed 266 out of 282 phosphosites.


,b1790p079_PP1_MCF7_AZD1480_r1,b1790p079_PP1_MCF7_AZD1480_r2,b1790p079_PP1_MCF7_AZD5363_r1,b1790p079_PP1_MCF7_AZD5363_r2,b1790p079_PP1_MCF7_BX_r1,b1790p079_PP1_MCF7_BX_r2,b1790p079_PP1_MCF7_CHIR_r1,b1790p079_PP1_MCF7_CHIR_r2,b1790p079_PP1_MCF7_DMSO_r1,b1790p079_PP1_MCF7_DMSO_r2,...,b1790p077_PP2_MCF7_KU_r1,b1790p077_PP2_MCF7_KU_r2,b1790p077_PP2_MCF7_PD_r1,b1790p077_PP2_MCF7_PD_r2,b1790p077_PP2_MCF7_PF_r1,b1790p077_PP2_MCF7_PF_r2,b1790p077_PP2_MCF7_TORIN_r1,b1790p077_PP2_MCF7_TORIN_r2,b1790p077_PP2_MCF7_TRAMETINIB_r1,b1790p077_PP2_MCF7_TRAMETINIB_r2
1A24_HUMAN(S356),2.692968e+08,350275208.7,2.339065e+08,147323791.5,2.292088e+08,1.992612e+08,222675485.3,203111064.7,1.724312e+08,1.952799e+08,...,194267298.8,244622442.6,122890720.9,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,460718561.8
1A24_HUMAN(S359),2.692968e+08,350275208.7,2.339065e+08,147323791.5,2.292088e+08,1.992612e+08,222675485.3,203111064.7,1.724312e+08,1.952799e+08,...,194267298.8,244622442.6,122890720.9,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,460718561.8
SYNDIG1(M1),7.695947e+08,523120228.8,1.278819e+09,701672838.7,6.317868e+07,3.288562e+08,767254583.2,886455926.9,3.021734e+09,1.579953e+09,...,0.0,0.0,527955514.4,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,165826164.4
SYNDIG1(M4),7.695947e+08,523120228.8,1.278819e+09,701672838.7,6.317868e+07,3.288562e+08,767254583.2,886455926.9,3.021734e+09,1.579953e+09,...,0.0,0.0,527955514.4,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,165826164.4
CRYBG1(S103),1.850112e+10,423470396.9,1.667692e+10,268909936.1,1.034383e+08,7.651674e+09,525139303.1,0.0,2.326042e+08,5.811915e+07,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,5.767428e+06,0.000000e+00,0.000000e+00,3.752771e+06,0.0


## 3. Mapping Protein IDs to HGNC Symbols Using HGNC (HGNC Multi-symbol checker)

The next crucial step is to align all gene names with the currently approved HGNC symbols. To achieve this, we leverage the HGNC Multi-ID Mapping Tool accessible via its API. This tool recognised outdated gene names or aliases and mapping them to their current, approved symbols.

By using this API, we not only update our dataset with the latest nomenclature but also retain information on the original state of each symbol, categorizing them as:
- Already approved (`Match type = symbol`)
- Successfully mapped from a previous name or alias (`Match type = prev_symbol`)
- Unmatched, indicating no current approved symbol could be found (`Match type = unmatched`).

In [8]:
# Utilize the HGNC API to map all gene names in our dataset to the current approved HGNC symbols.
# This includes detecting and updating previous gene names or aliases to their current approved forms.
hgnc_map = idm.get_hgnc_id_map(aup_m2.index, phosphosites=True, return_df=True)

# This DataFrame includes approved, mapped, and unmatched symbols.
hgnc_map.head()

INFO:root:Starting to fetch symbols for 4585 identifiers...


INFO:root:
Mapped 123 out of 4585 proteins to approved HGNC gene symbols.


,Input,Match type,Approved symbol
0,SYNDIG1,symbol,SYNDIG1
1,CFAP58,symbol,CFAP58
2,CRYBG1,symbol,CRYBG1
3,INTS13,symbol,INTS13
4,WDCP,symbol,WDCP


In [9]:
# Update the DataFrame with the HGNC-approved symbols.
# This step ensures that our dataset's gene names are standardized according to the latest nomenclature.
aup_m3 = idm.index_ids_to_hgnc_symbols(aup_m2, database='hgnc', phosphosites=True)

# Display the updated DataFrame to confirm the successful application of current approved HGNC symbols.
aup_m3.head()

INFO:root:Starting to fetch symbols for 4585 identifiers...


INFO:root:
Mapped 123 out of 4585 proteins to approved HGNC gene symbols.
INFO:root:Transformed 323 out of 14448 phosphosites.


,b1790p079_PP1_MCF7_AZD1480_r1,b1790p079_PP1_MCF7_AZD1480_r2,b1790p079_PP1_MCF7_AZD5363_r1,b1790p079_PP1_MCF7_AZD5363_r2,b1790p079_PP1_MCF7_BX_r1,b1790p079_PP1_MCF7_BX_r2,b1790p079_PP1_MCF7_CHIR_r1,b1790p079_PP1_MCF7_CHIR_r2,b1790p079_PP1_MCF7_DMSO_r1,b1790p079_PP1_MCF7_DMSO_r2,...,b1790p077_PP2_MCF7_KU_r1,b1790p077_PP2_MCF7_KU_r2,b1790p077_PP2_MCF7_PD_r1,b1790p077_PP2_MCF7_PD_r2,b1790p077_PP2_MCF7_PF_r1,b1790p077_PP2_MCF7_PF_r2,b1790p077_PP2_MCF7_TORIN_r1,b1790p077_PP2_MCF7_TORIN_r2,b1790p077_PP2_MCF7_TRAMETINIB_r1,b1790p077_PP2_MCF7_TRAMETINIB_r2
1A24_HUMAN(S356),2.692968e+08,350275208.7,2.339065e+08,147323791.5,2.292088e+08,1.992612e+08,222675485.3,203111064.7,1.724312e+08,1.952799e+08,...,194267298.8,244622442.6,122890720.9,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,460718561.8
1A24_HUMAN(S359),2.692968e+08,350275208.7,2.339065e+08,147323791.5,2.292088e+08,1.992612e+08,222675485.3,203111064.7,1.724312e+08,1.952799e+08,...,194267298.8,244622442.6,122890720.9,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,460718561.8
SYNDIG1(M1),7.695947e+08,523120228.8,1.278819e+09,701672838.7,6.317868e+07,3.288562e+08,767254583.2,886455926.9,3.021734e+09,1.579953e+09,...,0.0,0.0,527955514.4,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,165826164.4
SYNDIG1(M4),7.695947e+08,523120228.8,1.278819e+09,701672838.7,6.317868e+07,3.288562e+08,767254583.2,886455926.9,3.021734e+09,1.579953e+09,...,0.0,0.0,527955514.4,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,165826164.4
CRYBG1(S103),1.850112e+10,423470396.9,1.667692e+10,268909936.1,1.034383e+08,7.651674e+09,525139303.1,0.0,2.326042e+08,5.811915e+07,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,5.767428e+06,0.000000e+00,0.000000e+00,3.752771e+06,0.0


## 4. Manual Mapping of Unmatched Protein IDs to HGNC Symbols

After automating the mapping process as much as possible, our dataset now includes updated HGNC symbols. The next step involves identifying IDs that lack an approved HGNC gene symbol, so we can investigate and manually map them. We begin by identifying and exporting these unmatched IDs to a file. Subsequently, we incorporate a manually curated mapping of these IDs from an external source to update our dataframe.

In [10]:
# Filter for unmatched IDs from the HGNC mapping process.
# These IDs did not find a corresponding approved HGNC gene symbol and need manual investigation.
missing_hgnc_symbols = hgnc_map.loc[hgnc_map['Match type'] == 'unmatched', 'Input'].sort_values().reset_index(drop=True)

# Display the missing HGNC symbols for review.
print(missing_hgnc_symbols)

# Export the list of missing HGNC symbols to a TSV file for external manual mapping.
# This file serves as a reference for investigating and manually mapping these IDs.
missing_hgnc_symbols.to_csv('workspace/missing_hgnc_symbols.tsv', index=False, header=False)

0      1A24_HUMAN
1          AKT1.2
2     CJ012_HUMAN
3     FA21D_HUMAN
4     FRG1B_HUMAN
5          Sep-02
6          Sep-05
7          Sep-06
8          Sep-07
9          Sep-09
10    YA043_HUMAN
11    YJ005_HUMAN
12    YL004_HUMAN
13    YS060_HUMAN
14    YV023_HUMAN
Name: Input, dtype: object


In [11]:
# Load the manually mapped HGNC symbols from an external file.
# This file contains the mappings curated through external investigation.
mapped_hgnc_symbols = pd.read_csv('resources/external/mapped_hgnc_symbols.tsv', sep='\t')

# Create a mapping dictionary from the manually mapped symbols for conversion.
mapping_dict = dict(zip(mapped_hgnc_symbols['From'], mapped_hgnc_symbols['To']))

# Update the DataFrame with manually mapped HGNC symbols.
# This step integrates our manual mapping efforts into the dataset.
aup_m4 = idm.update_df_index_with_mappings(aup_m3, mapping_dict, phosphosites=True)

# Sort the updated DataFrame by index to organize the entries.
aup_m4.sort_index(inplace=True)

# Display the updated DataFrame with both automatically and manually mapped HGNC symbols.
aup_m4.head()

INFO:root:Transformed 27 out of 14448 phosphosites.


,b1790p079_PP1_MCF7_AZD1480_r1,b1790p079_PP1_MCF7_AZD1480_r2,b1790p079_PP1_MCF7_AZD5363_r1,b1790p079_PP1_MCF7_AZD5363_r2,b1790p079_PP1_MCF7_BX_r1,b1790p079_PP1_MCF7_BX_r2,b1790p079_PP1_MCF7_CHIR_r1,b1790p079_PP1_MCF7_CHIR_r2,b1790p079_PP1_MCF7_DMSO_r1,b1790p079_PP1_MCF7_DMSO_r2,...,b1790p077_PP2_MCF7_KU_r1,b1790p077_PP2_MCF7_KU_r2,b1790p077_PP2_MCF7_PD_r1,b1790p077_PP2_MCF7_PD_r2,b1790p077_PP2_MCF7_PF_r1,b1790p077_PP2_MCF7_PF_r2,b1790p077_PP2_MCF7_TORIN_r1,b1790p077_PP2_MCF7_TORIN_r2,b1790p077_PP2_MCF7_TRAMETINIB_r1,b1790p077_PP2_MCF7_TRAMETINIB_r2
AAAS(S495),2.664712e+09,3.635279e+09,3.285959e+09,2.743536e+09,3.822861e+09,4.393195e+09,3.287022e+09,3.540691e+09,2.247902e+09,6.012226e+09,...,8.963979e+09,8.351130e+09,9.330759e+09,7.767012e+09,7.900387e+09,7.211204e+09,8.041161e+09,8.921806e+09,8.499730e+09,7.352127e+09
AAGAB(M297),0.000000e+00,0.000000e+00,6.526145e+07,0.000000e+00,0.000000e+00,0.000000e+00,5.477754e+06,4.922887e+06,0.000000e+00,0.000000e+00,...,2.654259e+07,5.902313e+07,7.761711e+07,3.916353e+07,4.230814e+07,2.161106e+07,0.000000e+00,3.161788e+07,1.732876e+08,2.219852e+08
AAGAB(S310),1.219400e+09,1.578639e+09,1.083647e+09,9.467747e+08,1.850897e+08,3.065556e+08,1.231307e+08,6.337119e+07,2.275421e+08,8.498069e+07,...,7.082795e+08,8.271310e+08,5.338869e+08,3.855521e+08,9.326324e+08,6.311395e+08,6.821255e+08,5.115066e+08,9.902843e+08,7.665052e+08
AAGAB(S311),1.219400e+09,1.578639e+09,1.083647e+09,9.467747e+08,1.850897e+08,3.065556e+08,1.231307e+08,6.337119e+07,2.275421e+08,8.498069e+07,...,1.593237e+09,1.712089e+09,1.237525e+09,1.012566e+09,1.602475e+09,1.309217e+09,1.377144e+09,1.177742e+09,1.742970e+09,1.431783e+09
AAK1(S14),8.624913e+08,1.414261e+09,1.916668e+09,1.086577e+09,8.751045e+08,1.559491e+08,9.583288e+08,8.503202e+08,1.115720e+09,1.117170e+09,...,1.392150e+09,6.465040e+08,1.100319e+09,6.495936e+08,1.397288e+09,1.449160e+09,1.454729e+09,1.980154e+09,8.421942e+08,1.566950e+09


## Conclusion and Next Steps

In this notebook, we demonstrated a comprehensive workflow for mapping protein IDs to HGNC symbols using the UniProt database. This process is essential for standardizing protein identifiers in our dataset.

### Next Steps
Utilize the standardized dataset for further biological analysis, including:
- Data preprocessing,
- Quality control to identify and exclude low-quality samples and phosphosites
- Normalisation
- Differential phosphosite occupancy analysis (DPOA),
- Estimating missing fold changes,
- Calculating signal intensity-dependent p-values (sid-scores).

## References
- [UniProt - ID mapping](https://www.uniprot.org/id-mapping)
- [UniProtMapper](https://pypi.org/project/uniprot-id-mapper/)
- [UniProt - Search](https://www.uniprot.org/)
- [HGNC - Multi-symbol checker](https://www.genenames.org/tools/multi-symbol-checker/)